In [ ]:
# Packages

import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
import pytz
from datetime import timedelta

In [ ]:

RESPECK_FILE = '../data/bishkek_csr/03_train_ready/respeck/11-05-2025_respeck.csv'
PSG_FILE = '../data/bishkek_csr/03_train_ready/nasal_files/11-05-2025_nasal.csv'
LABELS_FILE = '../data/bishkek_csr/03_train_ready/event_exports/11-05-2025_event_export.csv'

respeck_df = pd.read_csv(RESPECK_FILE)
respeck_df['timestamp'] = pd.to_datetime(respeck_df['alignedTimestamp'], unit='ms')
tz = pytz.timezone('Asia/Bishkek')
respeck_df['timestamp'] = respeck_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
respeck_df.set_index('timestamp', inplace=True)

psg_df = pd.read_csv(PSG_FILE)
psg_df['timestamp'] = pd.to_datetime(psg_df['UnixTimestamp'], unit='ms')
tz = pytz.timezone('Asia/Bishkek')
psg_df['timestamp'] = psg_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
psg_df.set_index('timestamp', inplace=True)

labels_df = pd.read_csv(LABELS_FILE)
labels_df['timestamp'] = pd.to_datetime(labels_df['UnixTimestamp'], unit='ms')
tz = pytz.timezone('Asia/Bishkek')
labels_df['timestamp'] = labels_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
labels_df.set_index('timestamp', inplace=True)

In [ ]:
import plotly.graph_objects as go
import plotly.subplots as sp
from datetime import timedelta
import pandas as pd
import os # <<< ADDED >>> To manage file saving

# Assume respeck_df, psg_df, and osa_events_df are pre-loaded DataFrames

def plot_events(respeck_df, psg_df, osa_events_df, buffer_minutes=2):
    """
    Generates a SEPARATE plot for each event.
    Each plot contains three vertically stacked, x-axis-linked subplots for
    PSG nasal, PSG chest, and Respeck data.
    
    Parameters:
    - respeck_df: DataFrame with Respeck sensor data.
    - psg_df: DataFrame with PSG data (including 'Resp nasal' and 'Resp chest').
    - osa_events_df: DataFrame with event labels.
    - buffer_minutes: Minutes to show before and after each event.
    """
    
    event_type_name = "events"
    if not osa_events_df.empty and 'Event' in osa_events_df.columns:
        # Sanitize event type name for use in filenames
        event_type_name = osa_events_df['Event'].iloc[0].replace(" ", "_")

    if osa_events_df.empty:
        print(f"No '{event_type_name}' events found in the data.")
        return
    
    n_events = len(osa_events_df)
    print(f"Found {n_events} '{event_type_name}' events. Generating a separate plot for each...")
    
    # <<< ADDED >>> Create a directory to save the plots
    output_dir = f"{event_type_name}_plots"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Plots will be saved in the '{output_dir}/' directory.")
    
    buffer_td = timedelta(minutes=buffer_minutes)
    
    # <<< MODIFIED >>> The main loop now creates, shows, and saves a plot in each iteration
    for idx, (event_time, event_row) in enumerate(osa_events_df.iterrows()):
        
        event_display_name = event_row.get('Event', 'Event')
        print(f"\n--- Processing Event {idx+1}/{n_events} at {event_time} ---")

        # <<< MODIFIED >>> Create a new 3-row figure for THIS event
        fig = sp.make_subplots(
            rows=3, cols=1,
            subplot_titles=(
                "PSG Nasal Respiration", 
                "PSG Chest Respiration", 
                "Respeck Sensor Data"
            ),
            vertical_spacing=0.08 
        )
        
        start_time = event_time - buffer_td
        end_time = event_time + buffer_td
        
        duration = 30.0
        duration_val = event_row.get('Duration')
        if duration_val is not None:
            try:
                duration = float(str(duration_val).replace(',', '.'))
            except (ValueError, TypeError):
                print(f"Warning: Could not parse duration '{duration_val}'. Using default.")
        
        event_end_time = event_time + timedelta(seconds=duration)
        
        respeck_window = respeck_df[start_time:end_time]
        psg_window = psg_df[start_time:end_time]
        
        if respeck_window.empty and psg_window.empty:
            print(f"Skipping: No data found for event {idx+1} in the specified time window")
            continue
        
        # --- Plot Traces using static row numbers (1, 2, 3) ---
        
        # Plot PSG Nasal Respiration (Row 1)
        if not psg_window.empty and 'Resp nasal' in psg_window.columns:
            fig.add_trace(go.Scatter(
                x=psg_window.index, y=psg_window['Resp nasal'], mode='lines', name='PSG Nasal',
                line=dict(color='blue'), showlegend=False
            ), row=1, col=1)
        
        # Plot PSG Chest Respiration (Row 2)
        if not psg_window.empty and 'Resp chest' in psg_window.columns:
            fig.add_trace(go.Scatter(
                x=psg_window.index, y=psg_window['Resp chest'], mode='lines', name='PSG Chest',
                line=dict(color='purple'), showlegend=False
            ), row=2, col=1)

        # Plot Respeck Data (Row 3)
        if not respeck_window.empty:
            if 'breathingSignal' in respeck_window.columns:
                fig.add_trace(go.Scatter(
                    x=respeck_window.index, y=respeck_window['breathingSignal'], mode='lines',
                    name='Respeck Breathing', line=dict(color='orange', width=2), showlegend=True
                ), row=3, col=1)
            # Add other respeck signals if needed...
        
        # --- Add Event Markers to ALL THREE Plots ---
        for r in [1, 2, 3]:
            fig.add_vrect(
                x0=event_time, x1=event_end_time, fillcolor="red", opacity=0.2, 
                layer="below", line_width=0,
                annotation_text=f"{event_display_name} ({duration:.1f}s)" if r == 1 else "",
                annotation_position="top left", row=r, col=1
            )
            fig.add_vline(x=event_time, line_dash="dash", line_color="red", row=r, col=1)

        # --- Link and Update Axes ---
        fig.update_xaxes(range=[start_time, end_time], row=1, col=1)
        fig.update_xaxes(range=[start_time, end_time], row=2, col=1)
        fig.update_xaxes(range=[start_time, end_time], row=3, col=1)
        
        # Link all x-axes to the top one
        fig.update_xaxes(matches='x1', row=2, col=1)
        fig.update_xaxes(matches='x1', row=3, col=1)

        # Add title to the bottom x-axis
        fig.update_xaxes(title_text="Time", row=3, col=1)
        
        # --- Update Layout, Show, and Save the Figure ---
        fig.update_layout(
            height=800,
            title_text=f'<b>{event_display_name} Event #{idx+1}</b><br><sup>Time: {event_time.strftime("%Y-%m-%d %H:%M:%S")}</sup>',
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        
        # Show the plot in the notebook/IDE
        fig.show()
        
        # Save the plot to a unique HTML file
        filename = os.path.join(output_dir, f"event_{idx+1}.html")
        fig.write_html(filename)

    # --- Final Summary (outside the loop) ---
    print(f"\n=== Summary ===")
    print(f"Finished processing. Total events plotted: {n_events}")
    
    durations = pd.to_numeric(osa_events_df['Duration'].astype(str).str.replace(',', '.', regex=False), errors='coerce').dropna()
    if not durations.empty:
        print(f"Average event duration: {durations.mean():.1f} seconds")

In [ ]:

plot_events(respeck_df, psg_df, labels_df[labels_df['Event'] == 'Obstructive Apnea'], buffer_minutes=10)